In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool

class State(TypedDict):
    query: str

@tool
def perform_query(query: str):
    """:param query: The SQL query to be executed."""
    print(f"Performing query: {query}")
    return {"query":{query}}

def review_query(state: State):
    print(f"Reviewing query: {state['query']}")
    return state

In [2]:
builder = StateGraph(State)
builder.add_node("perform_query", perform_query)
builder.add_node("review_query", review_query)

builder.add_edge("review_query", "perform_query")
builder.add_edge(START, "review_query")
builder.add_edge("perform_query", END)

graph = builder.compile(interrupt_before=["perform_query"], checkpointer=MemorySaver())

In [4]:
intiial_input = {"query": "SELECT * FROM users"}
config = {"configurable": {"thread_id":"thread-1"}}

for event in graph.stream(intiial_input, config):
    print(event)
approval = input("Approve query execution? (yes/no): ")
if approval.lower() == "yes":
    for event in graph.stream(None, config):
        print(event)
else:
    print("Query execution cancelled.")

Reviewing query: SELECT * FROM users
{'review_query': {'query': 'SELECT * FROM users'}}
{'__interrupt__': ()}
Performing query: SELECT * FROM users
{'perform_query': {'query': {'SELECT * FROM users'}}}


In [5]:
from display_graph import display_graph
display_graph(graph)

d:\projects\github\learning\llm\langgraph\langgraph_blueprint\ch9/graphs/graph_72176.png
